# 네이버 영화 140자평 스크레이핑 프로그램 만들기

In [1]:
#! /usr/bin/python3
# -*- coding: utf-8 -*-

In [2]:
from os import makedirs
import os.path
import csv, codecs

import numpy as np
import pandas as pd
 
import urllib.request as req
import re # 정규식 관련 라이브러리
from bs4 import BeautifulSoup # html 파싱용 라이브러리

In [3]:
BASE_URL = "https://movie.naver.com/movie/point/af/list.nhn?target=after"
PAGE_URL = BASE_URL + "?&page=%s"

## 페이지 수 리턴 함수 정의

In [4]:
def count_page(url):
    html = req.urlopen(url)
    soup = BeautifulSoup(html, "html.parser")
    
    review_count = soup.select("#old_content > h5 > div > strong")
    review_count = int(review_count[0].string)
    
    return int(np.ceil(review_count / 10)) # 한 페이지당 10개의 게시글

## 파싱 함수 정의

In [5]:
def parse_item(item):
    # return 리뷰번호, 평점, 영화제목, 영화리뷰
    # num, point, title, review
    
    _num = item.find('td', {'class':'ac num'})
    _num = _num.text

    _point = item.find('td', {'class':'point'})
    _point = _point.text
    
    movie_info = item.find('td', {'class':'title'})
    movie_info = movie_info.text
    movie_info = movie_info.split('\n')
    
    _title = movie_info[1].strip()
    _review = movie_info[2].strip()
    
    try:
        return {'num': _num, 'point': _point, 'title': _title, 'review': _review}
    except (IndexError, AttributeError) as e:
        print(e, item.text)
    except (AssertionError) as e:
        print(e, "sleep for %s" % 600)
        time.sleep(600)
    except Exception as e:
        print(e)

In [6]:
def scraping_rating_page(url):   
    html = req.urlopen(url)
    soup = BeautifulSoup(html, 'html')
    
    items = soup.findAll('tr')
    items = items[1:] # 첫번째 행은 header 이므로 제거합니다.

    return list([parse_item(item) for item in items])

In [9]:
review_path = '../raw_data'

if not os.path.exists(review_path):
    print("make dirs=", review_path)
    makedirs(review_path)

## CSV 파일로 100 페이지 당 한번씩 Raw 데이터를 저장합니다.

In [25]:
review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))
review_file = '../raw_data/%s.csv'

for index in range(1, count_page(BASE_URL)):
    url = PAGE_URL % (index)
    add_list = scraping_rating_page(url)
    print(url)
    print(add_list)
    print('=============')
    print(review_df)
    
    
    review_df = review_df.append(add_list)
    
    if (index % 10) == 0:
        # 중복 row 제거
        
        #
        
        # 저장
        file_name = review_file % index
        review_df.to_csv(file_name, index=False, header=True)        
        print(file_name, '저장했습니다.')
        
        # 초기화
        review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))

https://movie.naver.com/movie/point/af/list.nhn?target=after?&page=1000
[{'num': '14759568', 'point': '10', 'title': '스타 이즈 본', 'review': '요즘볼게읍다 ㄹㅇㄹ ㅋㅋㅋ'}, {'num': '14759567', 'point': '6', 'title': '베놈', 'review': '현실성은 없지만 시간때우기로 볼만함'}, {'num': '14759566', 'point': '8', 'title': '서치', 'review': '몮ㄷㅈㄴㅇ재미있어요'}, {'num': '14759565', 'point': '10', 'title': '베놈', 'review': '진짜 개굴잼 꼭 보세요 여러분'}, {'num': '14759564', 'point': '10', 'title': '곰돌이 푸 다시 만나 행복해', 'review': '뭔가 살짝 허무한감도 있긴했었지만 내인생에 감동을 준 최고의 영화 :)'}, {'num': '14759563', 'point': '10', 'title': '베놈', 'review': 'ㅇㅇㅇㄴ재미있어요!'}, {'num': '14759562', 'point': '9', 'title': '베놈', 'review': '처음본저로서는 재밌어요! 뭔가 많이 기대하고가시면 생각보단 별로일수있어요'}, {'num': '14759561', 'point': '10', 'title': '호스틸', 'review': '6점으로 끝났을 좀비영화을 마지막 10분에 의해 10점이 되었다 최고의 좀비(?) 영화'}, {'num': '14759560', 'point': '10', 'title': '호밀밭의 반항아', 'review': '니콜라스 홀트 연기 최고였어요. 니콜라스 홀트는  사랑입니댜'}, {'num': '14759559', 'point': '10', 'title': '퍼스트맨', 'review': '2시간 20분의 긴 러닝타임동안 콜라를 다 못마시고

IndexError: list index out of range

In [17]:
review_df = pd.DataFrame(columns=('num', 'point', 'title', 'review'))
url = PAGE_URL % 1000
print(scraping_rating_page(url))
review_df = review_df.append(scraping_rating_page(url))

if (1000 % 10) == 0:
    print('저장')

[{'num': '14759560', 'point': '10', 'title': '호밀밭의 반항아', 'review': '니콜라스 홀트 연기 최고였어요. 니콜라스 홀트는  사랑입니댜'}, {'num': '14759559', 'point': '10', 'title': '퍼스트맨', 'review': '2시간 20분의 긴 러닝타임동안 콜라를 다 못마시고 나올정도로 몰입하게되는 영화중들중 하나'}, {'num': '14759558', 'point': '4', 'title': '협상', 'review': '노련한 협상가는 어떠한 경우에도 눈물을 보이지 않는다.'}, {'num': '14759555', 'point': '7', 'title': '안시성', 'review': '명량보단 나은듯...'}, {'num': '14759554', 'point': '10', 'title': '퍼스트맨', 'review': '나만 당할순없지 아이맥스 진짜 어휴 내돈'}, {'num': '14759553', 'point': '10', 'title': '퍼스트맨', 'review': '너무나 집중하며 봤네요.. 추천합니다 정말'}, {'num': '14759552', 'point': '10', 'title': '미쓰백', 'review': '어른들이 꼭 봐야할 영화입니다'}, {'num': '14759551', 'point': '9', 'title': '협상', 'review': '지루하지 않고, 연기도 좋았고, 너무 잔인한 장면도 아니고, 좋아요~'}, {'num': '14759550', 'point': '4', 'title': '곰돌이 푸 다시 만나 행복해', 'review': '후... 영화는 괜찮은데 전체이용가 영화라그런지 유아 어린이들 개많아서 난장판임ㅋ 힐링하러갓다가 혈압오름 나중에 vod 풀리면 그때보는거 추천'}, {'num': '14759549', 'point': '10', 'title': '그랜드 부다페스트 호텔', 'review': '영화 장면장면 너무 황홀했다.특히

In [15]:
review_df

,num,point,title,review
0,14759559,10,퍼스트맨,2시간 20분의 긴 러닝타임동안 콜라를 다 못마시고 나올정도로 몰입하게되는 영화중들...
1,14759558,4,협상,노련한 협상가는 어떠한 경우에도 눈물을 보이지 않는다.
2,14759555,7,안시성,명량보단 나은듯...
3,14759554,10,퍼스트맨,나만 당할순없지 아이맥스 진짜 어휴 내돈
4,14759553,10,퍼스트맨,너무나 집중하며 봤네요.. 추천합니다 정말
5,14759552,10,미쓰백,어른들이 꼭 봐야할 영화입니다
6,14759551,9,협상,"지루하지 않고, 연기도 좋았고, 너무 잔인한 장면도 아니고, 좋아요~"
7,14759550,4,곰돌이 푸 다시 만나 행복해,후... 영화는 괜찮은데 전체이용가 영화라그런지 유아 어린이들 개많아서 난장판임ㅋ ...
8,14759549,10,그랜드 부다페스트 호텔,"영화 장면장면 너무 황홀했다.특히 눈속을 달리는 장면과 멘들,호텔의 오묘하지만 조화..."
9,14759548,8,스타 이즈 본,"귀호강했는데,그렇게 보내냐!ㅜㅜ"
